In [1]:
!pip install --upgrade pip transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [2]:
import numpy as npimport pandas as pdimport osimport reimport transformersimport torchfrom sklearn.model_selection import train_test_splitfrom sklearn.utils.class_weight import compute_class_weightfrom transformers import XLMRobertaTokenizerfrom sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_reportfrom torch.utils.data import Datasetfrom transformers import ( XLMRobertaForSequenceClassification, XLMRobertaTokenizer, Trainer, TrainingArguments,)from peft import get_peft_model, LoraConfig, TaskTypefrom transformers.trainer_utils import get_last_checkpointfrom transformers import AutoTokenizerimport wandbfrom kaggle_secrets import UserSecretsClientfrom peft import PeftModel

2025-12-04 22:27:23.579224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764887243.969111      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764887244.081363      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
INPUT_DF_PATH = "/kaggle/input/depressive-data/depressive_data_with_entities.csv"INPUT_STOPWORDS_PATH = "/kaggle/input/stopwords-ru/stopwords-ru.txt"

In [4]:
df = pd.read_csv(INPUT_DF_PATH, encoding="utf-8")df.head()

/tmp/ipykernel_20/3621240148.py:1: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_DF_PATH, encoding="utf-8")
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in grea

,clean_text,label,age,text_length,word_count,language,entities,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45
0,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...,1,32.0,1699,272,ru,"[('–∞–ø–∞—Ç–∏—è', 'SYMPTOM')]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...,1,28.0,769,121,ru,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...,1,16.0,501,90,ru,"[('–ø–æ–∫–æ–Ω—á–∏—Ç—å —Å —Å–æ–±–æ–π', 'SUICIDA...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...,1,32.0,236,45,ru,"[('—É—Å—Ç–∞–ª', 'EMOTIONAL_STATE')]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...,1,14.0,2074,365,ru,"[('–¥–µ–ø—Ä–µ—Å—Å–∏—è', 'CONDITION'), ('–¥–µ–ø...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df[['clean_text', 'label']]df.head()

,clean_text,label
0,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...,1
1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...,1
2,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...,1
3,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...,1
4,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...,1


In [6]:
try: with open(INPUT_STOPWORDS_PATH, "r", encoding="utf-8") as f: stopwords_ru = [line.strip() for line in f if line.strip()]except FileNotFoundError: print(f"Error: The file '{file_path}' was not found. Please ensure the file exists in your Google Drive.")except Exception as e: print(f"An error occurred while loading the file: {e}")

In [7]:
def remove_stopwords(text, stop_words): """Removes stop words from a string.""" if isinstance(text, str): words = text.split() filtered_words = [word for word in words if word.lower() not in stop_words] return ' '.join(filtered_words) return ""# Apply stop word removal to the 'text' column of filtered_dfprint("Removing stop words from filtered_df...")df['text_no_stopwords'] = df['clean_text'].apply(lambda x: remove_stopwords(x, set(stopwords_ru)))print("DataFrame after removing stop words:")display(df.head())

Removing stop words from filtered_df...
DataFrame after removing stop words:


,clean_text,label,text_no_stopwords
0,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...,1,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...
1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...,1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...
2,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...,1,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...
3,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...,1,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...
4,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...,1,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...


In [8]:
# ConfigurationMODEL_NAME = "xlm-roberta-base"OUTPUT_DIR = "/kaggle/working/xlm-baseline-finetuned/"RANDOM_STATE = 42# Setup devicedevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')print(f" Using device: {device}")

✅ Using device: cuda


## Data Split

In [9]:
# Drop any rows with missing textdf.dropna()# If labels should be integers, convert or drop invalid rowsdf['label'] = pd.to_numeric(df['label'], errors='coerce')df.dropna(subset=['label'], inplace=True)df['label'] = df['label'].astype(int)# Split into train/validation and testtrain_val_df, test_df = train_test_split( df, test_size=0.15, random_state=RANDOM_STATE, stratify=df['label'])# Split train/validation into train and validationtrain_df, val_df = train_test_split( train_val_df, test_size=0.15, # 15% of the 85% random_state=RANDOM_STATE, stratify=train_val_df['label'])print(f"Training set size: {len(train_df)}")print(f"Validation set size: {len(val_df)}")print(f"Test set size: {len(test_df)}")

Training set size: 43432
Validation set size: 7665
Test set size: 9018


## Class Weights

In [10]:
# Get all labels from the training DataFrametrain_labels = train_df['label'].tolist()# Calculate weightsclass_weights = compute_class_weight( class_weight='balanced', classes=np.unique(train_labels), y=train_labels)# Convert to a PyTorch tensor on the GPUclass_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)print(f"Calculated class weights: {class_weights_tensor}")

Calculated class weights: tensor([0.9695, 1.0325], device='cuda:0')


## Tokenizer and Dataset

## Load The Model

In [11]:
# Configuration# The maximum sequence length for the model. Texts will be padded or truncated to this length.MAX_LENGTH = 256def normalize_text(text: str) -> str: """ Cleans and normalizes a string of text. This function performs the following steps: 1. Converts text to lowercase. 2. Removes URLs. 3. Removes non-alphanumeric characters (keeps basic punctuation for context). 4. Standardizes whitespace by replacing multiple spaces/newlines with a single space. Args: text: The input string to clean. Returns: The cleaned and normalized string. """ if not isinstance(text, str): return "" text = text.lower() # Remove URLs text = re.sub(r'https?://\S+|www\.\S+', '', text) # Remove HTML tags text = re.sub(r'<.*?>', '', text) # Keep letters, numbers, and basic punctuation, remove others text = re.sub(r'[^a-zа-я0-9\s.,!?-]', '', text) # Standardize whitespace text = re.sub(r'\s+', ' ', text).strip() return textdef process_dataframe(df: pd.DataFrame, text_column: str, label_column: str, tokenizer) -> pd.DataFrame: """ Cleans, tokenizes, and prepares a DataFrame for model input. Args: df: The input pandas DataFrame. text_column: The name of the column containing the text data. label_column: The name of the column containing the labels. tokenizer: An initialized Hugging Face tokenizer instance. Returns: A new DataFrame with cleaned text, original label, and tokenized input_ids and attention_mask. """ if text_column not in df.columns or label_column not in df.columns: print(f"Error: DataFrame must contain '{text_column}' and '{label_column}' columns. Skipping processing.") return pd.DataFrame() # Return empty DataFrame if columns are missing print(f"--- Processing DataFrame with text column '{text_column}' ---") # 1. Normalize Text print("Normalizing text...") df['clean_text'] = df[text_column].apply(normalize_text) # 2. Tokenize Text print("Tokenizing text (this may take a moment)...") encodings = tokenizer( df['clean_text'].tolist(), truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors='pt' # Returns PyTorch tensors ) # Add tokenized results and original label to a new DataFrame processed_df = pd.DataFrame({ 'clean_text': df['clean_text'], 'label': df[label_column], 'input_ids': encodings['input_ids'].tolist(), 'attention_mask': encodings['attention_mask'].tolist() }) print("Processing complete.") return processed_dfif __name__ == '__main__': # Initialize the tokenizer print("Initializing XLM-RoBERta tokenizer...") tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME) train_dataset = process_dataframe(train_df.copy(), 'text_no_stopwords', 'label', tokenizer) val_dataset = process_dataframe(val_df.copy(), 'text_no_stopwords', 'label', tokenizer) test_dataset = process_dataframe(test_df.copy(), 'text_no_stopwords', 'label', tokenizer)

Initializing XLM-RoBERta tokenizer...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

--- Processing DataFrame with text column 'text_no_stopwords' ---
Normalizing text...
Tokenizing text (this may take a moment)...
Processing complete.
--- Processing DataFrame with text column 'text_no_stopwords' ---
Normalizing text...
Tokenizing text (this may take a moment)...
Processing complete.
--- Processing DataFrame with text column 'text_no_stopwords' ---
Normalizing text...
Tokenizing text (this may take a moment)...
Processing complete.


In [12]:
# Load the XLM-RoBERTa tokenizertokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)class TextClassificationDataset(Dataset): def __init__(self, dataframe, tokenizer, max_length=256): self.tokenizer = tokenizer self.texts = dataframe['clean_text'].tolist() self.labels = dataframe['label'].tolist() self.max_length = max_length def __len__(self): return len(self.texts) def __getitem__(self, idx): text = self.texts[idx] label = self.labels[idx] encoding = self.tokenizer( text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt' ) return { 'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'labels': torch.tensor(label, dtype=torch.long) }# Create dataset instancestrain_dataset = TextClassificationDataset(train_dataset, tokenizer)val_dataset = TextClassificationDataset(val_dataset, tokenizer)test_dataset = TextClassificationDataset(test_dataset, tokenizer)

In [13]:
# 4. CONFIGURE MODEL AND PEFT (LoRA)print("\n Configuring PEFT (LoRA) model...")model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=df['label'].nunique())# LoRA is a form of adapter-tuning that is very memory-efficientpeft_config = LoraConfig( task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1, target_modules=['query', 'value'])model = get_peft_model(model, peft_config)model.print_trainable_parameters()


✅ Configuring PEFT (LoRA) model...


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 887,042 || all params: 278,932,228 || trainable%: 0.3180


In [14]:
# Move model to the GPUmodel.to(device)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): XLMRobertaForSequenceClassification(
      (roberta): XLMRobertaModel(
        (embeddings): XLMRobertaEmbeddings(
          (word_embeddings): Embedding(250002, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): XLMRobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x XLMRobertaLayer(
              (attention): XLMRobertaAttention(
                (self): XLMRobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
           

## Metrics and Custom Trainer

In [15]:
def compute_metrics(eval_pred): predictions, labels = eval_pred predictions = np.argmax(predictions, axis=1) # Get metrics per class (e.g., 0=control, 1=depression) precision, recall, f1, _ = precision_recall_fscore_support( labels, predictions, average=None, labels=[0, 1] ) # Get overall weighted metrics acc = accuracy_score(labels, predictions) weighted_f1 = precision_recall_fscore_support( labels, predictions, average='weighted' )[2] # [2] is the f1-score return { 'accuracy': acc, 'f1_weighted': weighted_f1, 'f1_control': f1[0], 'f1_depression': f1[1] # The metric we want to optimize }class WeightedLossTrainer(Trainer): # Add **kwargs to accept the new num_items_in_batch argument def compute_loss(self, model, inputs, return_outputs=False, **kwargs): labels = inputs.pop("labels") outputs = model(**inputs) logits = outputs.get("logits") # Manually compute the loss using our class weights loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor) loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1)) return (loss, outputs) if return_outputs else loss

## Train The Model

In [16]:
# 5. SET TRAINING ARGUMENTSprint("\n Setting Training Arguments...")training_args = TrainingArguments( output_dir=OUTPUT_DIR, num_train_epochs=3, per_device_train_batch_size=16, per_device_eval_batch_size=32, warmup_steps=500, weight_decay=0.01, logging_dir=f"{OUTPUT_DIR}/logs", logging_steps=1000, eval_strategy="steps", eval_steps=10000, save_strategy="steps", save_steps=10000, load_best_model_at_end=True, metric_for_best_model="loss", greater_is_better=False, fp16=True, # Use mixed precision for faster training on GPUs save_total_limit=2 # Only keep the 3 best/most recent checkpoints)


✅ Setting Training Arguments...


In [17]:
# Retrieve the secret key and loginuser_secrets = UserSecretsClient()wandb_api_key = user_secrets.get_secret("wandb_api_key")wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alakbarlic (alakbarlic-ada-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [18]:
# 6. INITIALIZE AND START TRAINERtrainer = Trainer( model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset)print("\n Starting trainer...")# CHECKPOINT_INPUT_DIR = "/kaggle/input/checkpoints/checkpoint-4074"# # Check if a last checkpoint exists in the INPUT directory# last_checkpoint = get_last_checkpoint(CHECKPOINT_INPUT_DIR)last_checkpoint = None


🚀 Starting trainer...


In [19]:
# if last_checkpoint is not None:# print(f" Checkpoint found at {last_checkpoint}. Resuming training...")# # This resumes from the OLD checkpoint, but saves NEW ones to OUTPUT_DIR# trainer.train(resume_from_checkpoint=last_checkpoint)# else:# print(" No checkpoint found. Starting training from scratch...")# trainer.train()# print("\n Training complete!")

## Evaluate and Save

In [20]:
print(" Running predictions on the test set...")prediction_output = trainer.predict(test_dataset)# Print the metricsprint("\n--- Test Metrics from Trainer ---")print(prediction_output.metrics)# Generate and save the classification reportlogits = prediction_output.predictionspredictions = np.argmax(logits, axis=1)true_labels = prediction_output.label_ids# (Assuming 0='control', 1='depression')report = classification_report( true_labels, predictions, target_names=['control', 'depression'])print("\n--- Detailed Classification Report ---")print(report)# Save the final reportreport_path = os.path.join(OUTPUT_DIR, "final_classification_report.txt")with open(report_path, "w") as f: f.write(report)# Save the final model and tokenizertrainer.save_model(os.path.join(OUTPUT_DIR, "best_model"))tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "best_model"))print(f" Best model and report saved to {OUTPUT_DIR}")

🚀 Running predictions on the test set...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251204_223008-pw9rbg8g
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deft-leaf-43
wandb: ⭐️ View project at https://wandb.ai/alakbarlic-ada-university/huggingface
wandb: 🚀 View run at https://wandb.ai/alakbarlic-ada-university/huggingface/runs/pw9rbg8g



--- Test Metrics from Trainer ---
{'test_loss': 0.69856196641922, 'test_runtime': 73.4517, 'test_samples_per_second': 122.775, 'test_steps_per_second': 1.92}

--- Detailed Classification Report ---
              precision    recall  f1-score   support

     control       0.39      0.07      0.11      4651
  depression       0.47      0.89      0.62      4367

    accuracy                           0.46      9018
   macro avg       0.43      0.48      0.36      9018
weighted avg       0.43      0.46      0.36      9018

✅ Best model and report saved to /kaggle/working/xlm-baseline-finetuned/
